# Python decorators and some applications

### to run this notebook
```bash
pip install jupyter # sudo if necessary
jupyter notebook # in the directory that contains this notebook
```
When you want to add two numbers and double the output

In [1]:
def add(n1, n2):
    return n1 + n2


def add_and_print(n1, n2):
    print n1, n2
    s = n1 + n2
    print s
    return s
    

print add(1, 2)

add_and_print(1, 2)


3
1 2
3


3

## Everything in python is object
### A function can return another function
### You can define a function in the block a function

In [2]:
def print_inout(whatever=None):
    
    def add_and_print(n1, n2):
        print n1, n2
        s = n1 + n2
        print s
        return s
    
    return add_and_print

add_and_print = print_inout()

add_and_print(1, 2)

print_inout()(1, 2)

1 2
3
1 2
3


3

### Function can be an input parameter too
`double` takes an input parameter `the_add_func`

In [3]:
def print_inout(the_add_func):
    
    def add_and_print(n1, n2):
        print n1, n2
        s = the_add_func(n1, n2)
        print s
        return s
    
    return add_and_print

add_and_print = print_inout(add)

add_and_print(1, 2)

1 2
3


3

## How about taking an arbitrary function

In [4]:
def print_inout(whatever_returns_something):
    
    def whatever_and_print(*args, **kwargs):
        print args, kwargs
        s = whatever_returns_something(*args, **kwargs)
        print s
        return s
    
    return whatever_and_print

add_and_print = print_inout(add)

add_and_print(1, 2)

(1, 2) {}
3


3

## And we usually write it this way...

In [5]:
def print_inout(func):
    
    def inner(*args, **kwargs):
        print 'function input: ', args, kwargs
        s = func(*args, **kwargs)
        print 'function output:', s
        return s
    
    return inner

## The `@` symbol

In [6]:
def add(n1, n2):
    return n1 + n2

add = print_inout(add)
add(1, 2)

@print_inout
def subtract(n1, n2):
    return n1 - n2

subtract(6, 2)

@print_inout
def foo(n1, n2, n3, str1):
    return 'nothing'

foo(1, set(), n3=0, str1='bar')

function input:  (1, 2) {}
function output: 3
function input:  (6, 2) {}
function output: 4
function input:  (1, set([])) {'str1': 'bar', 'n3': 0}
function output: nothing


'nothing'

## A real world example

In [7]:
import time

def timeit(func):
    def inner(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - start
        print '%s takes %.4fs to finish' % (func.__name__, elapsed)
        return result
    
    return inner

@timeit
def dummy_loop():
    for i in xrange(1000000):
        1.2 * 3.4 / 5.6
    return 0

dummy_loop()

dummy_loop takes 0.0503s to finish


0


## why my funtion signature is changed

In [10]:
@timeit
def foo(bar=0):
    """
    takes an input parameter 'bar'
    this function does nothing
    """
    return bar

print foo
help(foo)

<function foo at 0x10477f6e0>
Help on function foo in module __main__:

foo(*args, **kwargs)
    takes an input parameter 'bar'
    this function does nothing



## use the builtin `functools.wraps`

In [9]:
from functools import wraps

def timeit(func):
    @wraps(func) # <---- this is the magic
    def inner(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - start
        print '%s takes %.4fs to finish' % (func.__name__, elapsed)
        return result
    
    return inner

## A dynamic decorator

In [11]:
def timeit(unit='s'):
    assert(unit in ['s', 'ms'])
    def _timeit(func):
        @wraps(func)
        def inner(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            elapsed = time.time() - start
            if unit == 'ms':
                elapsed *= 1000
            print '%s takes %.4f%s to finish' % (func.__name__, elapsed, unit)
            return result
        
        return inner # <-- returns a decorated function
    
    return _timeit # <-- returns a new decorator

@timeit()
def dummy_loop1():
    for i in xrange(1000000): 
        1.2 * 3.4 / 5.6
    return 0

dummy_loop1()

@timeit('ms')
def dummy_loop2():
    for i in xrange(1000000): 
        1.2 * 3.4 / 5.6
    return 0

dummy_loop2()

@timeit(unit='minute')
def dummy_loop3():
    for i in xrange(1000000): 
        1.2 * 3.4 / 5.6
    return 0

dummy_loop3()

dummy_loop1 takes 0.0484s to finish
dummy_loop2 takes 45.3060ms to finish


AssertionError: 

## order matters

In [12]:
def double(func):
    def inner(*args, **kwargs):
        return func(*args, **kwargs) * 2
    
    return inner

def plus_ten(func):
    def inner(*args, **kwargs):
        return func(*args, **kwargs) + 10
    
    return inner

@double
@plus_ten
def f1():
    return 1

@plus_ten
@double
def f2():
    return 1

print f1()
print f2()

22
12


   ## decorators dynamically alter the functionality of a function or method
   ### django transaction atomic

In [ ]:
@transaction.atomic
def view1(request):
    return Response()

def view2(request):
    with transaction.atomic():
        return Response()

# the source code
def atomic(using=None, savepoint=True):
    # Bare decorator: @atomic -- although the first argument is called
    # `using`, it's actually the function being decorated.
    if callable(using):
        return Atomic(DEFAULT_DB_ALIAS, savepoint)(using)
    # Decorator: @atomic(...) or context manager: with atomic(...): ...
    else:
        return Atomic(using, savepoint)

   ### implementing a simple async taskqueue using hosted mq service
   this is only for demonstration purpose. You need to implement your own queue and message for it to work properly

In [ ]:
# taskqueue.py
import json

if queue == 'alibaba':
    from ali import queue, Message
elif queue == 'aws':
    from aws import queue, Message
elif queue == 'google':
    from gcp import queue, Message

class Tasklet(object):
    def __init__(self, payload):
        self.msg = Message(payload)
        
    def delay(self, seconds=0):
        if seconds > 0:
            self.msg.set_delayseconds(seconds)
        reply = queue.enqueue(self.msg) # send to the mq on cloud
        log.info('task_sent')
        return reply


def async(func):
    @wraps(func)
    def inner(*args, **kwargs):
        # _exec controls whether to return a tasklet or to execute the function
        if kwargs.pop('_exec', False):
            return func(*args, **kwargs)
        data = {'args': args, 'kwargs': kwargs, 'task': func.__name__}
        # json.dumps might fail, change to pickle when necessary
        # but in general it's better to avoid complex object passing externally
        payload = json.dumps(data, ensure_bytes=True)
        task = Tasklet(payload)
        return task
    
    return inner


# tasks/__init__.py
from taskqueue import async

@async
def ping():
    print 'pong'

    
# views.py
from tasks import ping

def view(request):
    ping().delay(3)


# worker.py
import importlib
from taskqueue import queue
import time
import json

_imported_modules = None

def run():
    while True:
        time.sleep(0.1)
        try:
            message = queue.receive()
        except:
            continue
        
        payload = json.loads(message)
        func = getattr(_imported_modules, payload['task'], None)
        if func is None:
            continue
        
        try:
            func(_exec=True, *payload['args'], **payload['kwargs'])
        except:
            continue

if __name__ == '__main__':
    _imported_modules = importlib.import_modules('tasks')
    run()


by Mengchi @ [Magnum Research Limited](https://www.aqumon.com)  
If you have passion in developing and FinTech, we're hiring! Send your CV to [career@magnumwm.com](career@magnumwm.com)